# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 8:30AM,244107,10,MSP213078,"Methapharm, Inc.",Released
1,Aug 15 2022 8:30AM,244107,10,MSP213079,"Methapharm, Inc.",Released
2,Aug 15 2022 8:30AM,244107,10,MSP213080,"Methapharm, Inc.",Released
3,Aug 15 2022 8:30AM,244107,10,MSP213081,"Methapharm, Inc.",Released
4,Aug 15 2022 8:30AM,244107,10,MSP213082,"Methapharm, Inc.",Released
5,Aug 15 2022 8:30AM,244107,10,MSP213083,"Methapharm, Inc.",Released
6,Aug 15 2022 8:30AM,244107,10,MSP213084,"Methapharm, Inc.",Released
7,Aug 15 2022 8:30AM,244107,10,MSP213088,"Methapharm, Inc.",Released
8,Aug 15 2022 8:30AM,244107,10,MSP213089,"Methapharm, Inc.",Released
9,Aug 15 2022 8:30AM,244107,10,MSP213090,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,244108,Released,11
19,244109,Released,27
20,244110,Released,15
21,244111,Released,1
22,244113,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244108,NaN,NaN,11.0
244109,NaN,NaN,27.0
244110,NaN,NaN,15.0
244111,NaN,NaN,1.0
244113,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244051,0.0,0.0,1.0
244056,0.0,0.0,1.0
244059,0.0,1.0,0.0
244060,0.0,0.0,1.0
244061,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244051,0,0,1
244056,0,0,1
244059,0,1,0
244060,0,0,1
244061,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244051,0,0,1
1,244056,0,0,1
2,244059,0,1,0
3,244060,0,0,1
4,244061,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244051,,,1
1,244056,,,1
2,244059,,1,
3,244060,,,1
4,244061,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc."
10,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation
21,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation
48,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation
63,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health
64,Aug 12 2022 5:07PM,244113,16,"Emersa Waterbox, LLC"
65,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.
78,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America
79,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc."
80,Aug 12 2022 2:03PM,244090,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",,,10
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,,,11
2,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,,,27
3,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,,,15
4,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,,,1
5,Aug 12 2022 5:07PM,244113,16,"Emersa Waterbox, LLC",,,1
6,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.,,,13
7,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,,,1
8,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",,,1
9,Aug 12 2022 2:03PM,244090,21,"NBTY Global, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,,
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,,
2,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27,,
3,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,15,,
4,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,1,,
5,Aug 12 2022 5:07PM,244113,16,"Emersa Waterbox, LLC",1,,
6,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.,13,,
7,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1,,
8,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",1,,
9,Aug 12 2022 2:03PM,244090,21,"NBTY Global, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,,
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,,
2,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27,,
3,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,15,,
4,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10.0,NaN,NaN
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11.0,NaN,NaN
2,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27.0,NaN,NaN
3,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,15.0,NaN,NaN
4,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10.0,0.0,0.0
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11.0,0.0,0.0
2,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27.0,0.0,0.0
3,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,15.0,0.0,0.0
4,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1952791,90.0,0.0,0.0
16,1464400,5.0,1.0,0.0
20,1220409,53.0,3.0,25.0
21,488183,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1952791,90.0,0.0,0.0
1,16,1464400,5.0,1.0,0.0
2,20,1220409,53.0,3.0,25.0
3,21,488183,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,90.0,0.0,0.0
1,16,5.0,1.0,0.0
2,20,53.0,3.0,25.0
3,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,90.0
1,16,Released,5.0
2,20,Released,53.0
3,21,Released,1.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,20,21
Status,,,,
Completed,0.0,0.0,25.0,0.0
Executing,0.0,1.0,3.0,1.0
Released,90.0,5.0,53.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,20,21
0,Completed,0.0,0.0,25.0,0.0
1,Executing,0.0,1.0,3.0,1.0
2,Released,90.0,5.0,53.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,20,21
0,Completed,0.0,0.0,25.0,0.0
1,Executing,0.0,1.0,3.0,1.0
2,Released,90.0,5.0,53.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()